<a href="https://colab.research.google.com/github/Artorio117/Patolog-a-en-plantas/blob/main/Conv2D_no_fitting_prueba05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [1]:
import os
from os import listdir
from os.path import isfile,isdir, join


from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import keras
import tensorflow as tf
from keras.models import load_model

from keras.preprocessing import image
from keras import layers, models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD

#pip install sklearn
#pip install scikit-image
from sklearn.model_selection import train_test_split
#from skimage import color, io
#from scipy.misc import imresize  ## deprecated
from PIL import Image

import datetime

In [3]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json 

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle competitions download -c plant-pathology-2021-fgvc8

 76% 11.4G/14.9G [01:29<00:19, 189MB/s]

In [ ]:
! unzip plant-pathology-2021-fgvc8

In [5]:
#Tamaño de la imagen
ih, iw = 120, 120

#Cuantas veces entrenar
epochs = 30


# Batch para hacer cada entrenamiento.
batch_size = 50 #1


# Número de imagenes en train
num_train = 16769 #18632
#Número de imagenes en test
num_test = 1863


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


#Carga del archivo
etiquetas = pd.read_csv('train.csv')
etiquetas["labels"] = etiquetas["labels"].apply(lambda x:x.split(" "))

image_generator = ImageDataGenerator(rescale = 1. / 255,
                                     validation_split=0.1)

train_ds = image_generator.flow_from_dataframe(etiquetas,directory = 'train_images',
                                             x_col = 'image',
                                             y_col = 'labels',
                                             batch_size=50,
                                             target_size=(iw, ih),
                                             subset='training',
                                             class_mode = 'categorical',
                                             classes=['healthy', 'rust',
                                                      'scab', 'complex',
                                                      'frog_eye_leaf_spot',
                                                      'powdery_mildew'])

test_ds = image_generator.flow_from_dataframe(etiquetas,directory = 'train_images',
                                             x_col = 'image',
                                             y_col = 'labels',
                                             batch_size=50,
                                             target_size=(iw, ih),
                                             subset='validation',
                                             class_mode = 'categorical',
                                             classes=['healthy', 'rust',
                                                      'scab', 'complex',
                                                      'frog_eye_leaf_spot',
                                                      'powdery_mildew'])



#para cargar pesos de la red desde donde se quedó la ultima vez
#filename = "cvsd.h5"
#model.load_weights(filename)  #comentar si se comienza desde cero.
###


Found 16769 validated image filenames belonging to 6 classes.
Found 1863 validated image filenames belonging to 6 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(10, (3, 3), input_shape=(ih, iw,3))) #50, 50
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(10, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(20, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tbCallBack = keras.callbacks.TensorBoard(log_dir=log_dir,
                                         histogram_freq=1, 
                                         write_graph=True, 
                                         write_images=True)
#python -m tensorboard.main --logdir=/Graph  <- Para correr Tensor board
#tensorboard  --logdir Graph/
print('Logs:')
print(log_dir)
print('______________________________________________________________')


from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("Conv2D_no_fittin_prueba05.h5",
                             monitor='val_accuracy', verbose=1,
                             save_best_only=True, save_weights_only=False,
                             save_freq='epoch', mode='auto')

history = model.fit_generator(
                train_ds,
                epochs=epochs,
                validation_data=test_ds,
                validation_steps=test_steps,
                callbacks=[tbCallBack, checkpoint]
                )


Logs:
logs/fit/20220513-160029
______________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
336/336 [==============================] - ETA: 0s - loss: 0.4535 - accuracy: 0.2889
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to Conv2D_no_fittin_prueba05.h5
336/336 [==============================] - 2498s 7s/step - loss: 0.4535 - accuracy: 0.2889 - val_loss: 0.3956 - val_accuracy: 0.4000
Epoch 2/30
336/336 [==============================] - ETA: 0s - loss: 0.4117 - accuracy: 0.3774
Epoch 2: val_accuracy improved from 0.40000 to 0.47081, saving model to Conv2D_no_fittin_prueba05.h5
336/336 [==============================] - 2485s 7s/step - loss: 0.4117 - accuracy: 0.3774 - val_loss: 0.3755 - val_accuracy: 0.4708
Epoch 3/30
336/336 [==============================] - ETA: 0s - loss: 0.3874 - accuracy: 0.4312
Epoch 3: val_accuracy improved from 0.47081 to 0.53189, saving model to Conv2D_no_fittin_prueba05.h5
336/336 [==============================] - 2483s 7s/step - loss: 0.3874 - accuracy: 0.4312 - val_loss: 0.3492 - val_accuracy: 0.5319
Epoch 4/30
33